# 1 导入库和设置

In [35]:
import pandas as pd
import jieba
import jieba.analyse
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
import json
from IPython.display import display, Markdown
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False

# 准备停用词
if not os.path.exists('cn_stopwords.txt'):
    basic_stopwords = """的 了 在 是 我 有 和 就 不 人 都 一个 也 很 到 说 要 去 你 会 着 没有 看 好 自己 这 上 我们""".split()
    with open('stopwords.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(basic_stopwords))
    print("已创建基础停用词文件 stopwords.txt")

with open('cn_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = set([line.strip() for line in f if line.strip()])

# 2 数据加载和验证函数

In [36]:
def load_and_validate_data(file_paths):
    """加载并验证数据"""
    all_datasets = {}
    problematic_files = []
    
    for file_path in file_paths:
        try:
            # 提取数据集名称
            dataset_name = os.path.splitext(os.path.basename(file_path))[0]
            
            # 读取Excel文件
            df = pd.read_excel(file_path)
            
            # 检查必要列是否存在
            if '字段1' not in df.columns:
                problematic_files.append((file_path, f"缺少'字段1'列"))
                continue
                
            # 检查数据是否为空
            if df.empty:
                problematic_files.append((file_path, "数据为空"))
                continue
                
            # 添加到数据集
            all_datasets[dataset_name] = df
            print(f"成功加载: {file_path} (记录数: {len(df)})")
            
        except Exception as e:
            problematic_files.append((file_path, f"加载错误: {str(e)}"))
    
    # 打印问题文件信息
    if problematic_files:
        print("\n问题文件:")
        for file, reason in problematic_files:
            print(f"- {file}: {reason}")
    
    return all_datasets

# 文件路径设置
file_paths = [
    'raw_data\阿里健康大药房.xlsx',
    'raw_data\福士德.xlsx', 
    'raw_data\耀典.xlsx',
    'raw_data\hfine.xlsx'
]

# 加载数据
all_datasets = load_and_validate_data(file_paths)

# 显示加载的数据集信息
if all_datasets:
    print("\n已加载数据集:")
    for name, df in all_datasets.items():
        print(f"- {name}: {len(df)}条记录")
else:
    print("没有有效数据集被加载")

成功加载: raw_data\阿里健康大药房.xlsx (记录数: 1044)
成功加载: raw_data\福士德.xlsx (记录数: 391)
成功加载: raw_data\耀典.xlsx (记录数: 319)
成功加载: raw_data\hfine.xlsx (记录数: 423)

已加载数据集:
- 阿里健康大药房: 1044条记录
- 福士德: 391条记录
- 耀典: 319条记录
- hfine: 423条记录


# 3 分数据集去重函数

In [40]:
import re  # 新增导入

In [41]:
def deduplicate_dataset(df, dataset_name):
    """对单个数据集执行去重"""
    print(f"\n=== 正在处理数据集: {dataset_name} ===")
    print(f"原始记录数: {len(df)}")
    
    # 1. 完全重复去重
    df = df.drop_duplicates(subset=['字段1'], keep='first')
    print(f"完全去重后: {len(df)} 条")
    
    # 2. 无效评论过滤
    invalid_patterns = ['嘻嘻', '哈哈', '好评', '。。。', '！！', r'\?\?', r'\*\*', '路过', '看看']
    
    # 构建正则表达式模式，确保特殊字符被转义
    pattern = '|'.join([re.escape(p) for p in invalid_patterns])
    
    # 确保'字段1'是字符串类型
    df['字段1'] = df['字段1'].astype(str)
    
    mask = (
        (df['字段1'].str.len() >= 5) &  # 长度大于等于5
        ~df['字段1'].str.contains(pattern, na=False, regex=True)  # 不包含无效模式
    )
    df = df[mask].copy()
    print(f"无效过滤后: {len(df)} 条")
    
    # 3. 相似评论去重（仅对小于500条的数据集执行）
    if len(df) > 0 and len(df) <= 500:
        try:
            texts = df['字段1'].tolist()
            vectorizer = TfidfVectorizer(tokenizer=jieba.cut, stop_words=list(stopwords))
            tfidf = vectorizer.fit_transform(texts)
            cos_sim = cosine_similarity(tfidf)
            
            duplicates = set()
            for i in range(len(cos_sim)):
                if i not in duplicates:
                    similar_indices = [j for j in range(i+1, len(cos_sim)) 
                                     if cos_sim[i][j] >= 0.85]
                    duplicates.update(similar_indices)
            
            df = df.drop(index=list(duplicates)).copy()
            print(f"相似去重后: {len(df)} 条")
        except Exception as e:
            print(f"相似去重时出错: {str(e)}，跳过此步骤")
    elif len(df) > 500:
        print("数据集较大(>500条)，跳过相似度去重")
    
    return df

# 4 去重结果验证

In [43]:
# 创建输出目录
os.makedirs('analysis_results', exist_ok=True)

# 对每个数据集执行去重
cleaned_datasets = {}
for name, df in all_datasets.items():
    try:
        # 确保数据是DataFrame且包含'字段1'
        if not isinstance(df, pd.DataFrame) or '字段1' not in df.columns:
            print(f"数据集 {name} 格式不正确，跳过")
            continue
            
        # 执行去重
        cleaned_df = deduplicate_dataset(df.copy(), name)
        cleaned_datasets[name] = cleaned_df
        
        # 保存结果
        output_path = f'analysis_results/{name}_cleaned.xlsx'
        cleaned_df.to_excel(output_path, index=False)
        print(f"已保存到: {output_path}\n")
        
    except Exception as e:
        print(f"处理数据集 {name} 时出错: {str(e)}")
        continue

# 汇总统计
if cleaned_datasets:
    print("\n=== 去重汇总 ===")
    for name in all_datasets.keys():
        if name in cleaned_datasets:
            orig_count = len(all_datasets[name])
            cleaned_count = len(cleaned_datasets[name])
            reduction = (orig_count - cleaned_count) / orig_count if orig_count > 0 else 0
            print(f"{name}: {orig_count} → {cleaned_count} (减少{reduction:.1%})")
else:
    print("没有执行任何去重操作")


=== 正在处理数据集: 阿里健康大药房 ===
原始记录数: 1044
完全去重后: 936 条
无效过滤后: 798 条
数据集较大(>500条)，跳过相似度去重
已保存到: analysis_results/阿里健康大药房_cleaned.xlsx


=== 正在处理数据集: 福士德 ===
原始记录数: 391
完全去重后: 366 条
无效过滤后: 315 条
相似去重后: 310 条
已保存到: analysis_results/福士德_cleaned.xlsx


=== 正在处理数据集: 耀典 ===
原始记录数: 319
完全去重后: 303 条
无效过滤后: 232 条
相似去重后: 230 条
已保存到: analysis_results/耀典_cleaned.xlsx


=== 正在处理数据集: hfine ===
原始记录数: 423
完全去重后: 414 条
无效过滤后: 360 条
相似去重后: 358 条
已保存到: analysis_results/hfine_cleaned.xlsx


=== 去重汇总 ===
阿里健康大药房: 1044 → 798 (减少23.6%)
福士德: 391 → 310 (减少20.7%)
耀典: 319 → 230 (减少27.9%)
hfine: 423 → 358 (减少15.4%)


C:\Users\avawa\AppData\Local\Temp\ipykernel_4700\1241546093.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['字段1'] = df['字段1'].astype(str)
C:\Users\avawa\AppData\Local\Temp\ipykernel_4700\1241546093.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['字段1'] = df['字段1'].astype(str)
c:\Users\avawa\.conda\envs\statmodel\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\avawa\

In [45]:
def show_deleted_examples(original_name):
    """显示被删除的评论示例"""
    if original_name not in all_datasets or original_name not in cleaned_datasets:
        print(f"数据集 {original_name} 不存在")
        return
    
    original = all_datasets[original_name]
    cleaned = cleaned_datasets[original_name]
    
    # 找出被删除的记录
    merged = original.merge(
        cleaned, 
        how='left', 
        indicator=True, 
        on=list(original.columns)
    )
    deleted = merged[merged['_merge'] == 'left_only']
    
    print(f"\n数据集: {original_name}")
    print(f"删除记录数: {len(deleted)}")
    
    if not deleted.empty:
        print("\n被删除的评论示例:")
        display(deleted.sample(min(5, len(deleted)))[['字段1']])
    else:
        print("\n没有删除任何评论")

# 交互式验证
if cleaned_datasets:
    print("可用数据集:", list(cleaned_datasets.keys()))
    dataset_to_check = input("输入要验证的数据集名称: ")
    show_deleted_examples(dataset_to_check)
else:
    print("没有可验证的去重结果")

可用数据集: ['阿里健康大药房', '福士德', '耀典', 'hfine']

数据集: 福士德
删除记录数: 81

被删除的评论示例:


,字段1
85,非常好
76,非常好用
81,做工精致
144,很轻便
104,非常好
